In [18]:
import networkx as nx
import numpy as np
import dgl

C:\Users\Esysss\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Esysss\AppData\Local\Programs\Python\Python310\lib\site-packages\dgl\backend\pytorch\tensor.py:31: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  return th.device('cpu')


In [19]:
def bipartite(T_train):
    user, item = np.nonzero(T_train)

    # G = nx.Graph()
    G = nx.DiGraph()
    for i, j in zip(user, item):
        G.add_edge("user"+ str(i) ,"item"+ str(j), weight=T_train[i,j])

    return G

In [20]:
train = np.load('train.npy')
test = np.load('test.npy')

T = train + test

In [21]:
graph = bipartite(T)

g = dgl.from_networkx(graph, edge_attrs=['weight'])
g.ndata['feat'] = torch.rand(len(graph.nodes()), 5)

import pickle
file = open('dpp.p','wb')
pickle.dump(g.edata,file)
file.close()

"""## Graph Visualization

We use the networkx library for visualizing the friendship relationships. There are hundreds of nodes (not shown), and thousands of links, so it can seem a bit messy when the graph is drawn.
"""

TEST_RATIO = 0.1  # ratio for splitting the test set

u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * TEST_RATIO)  # number of edges in test set
train_size = g.number_of_edges() - test_size  # number of edges in train set

# get positive edges for test and train
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges
adj = nx.adjacency_matrix(graph)
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

# split the negative edges for training and testing 
neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

# construct positive and negative graphs for training and testing
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

# training graph
train_g = dgl.remove_edges(g, eids[:test_size])
train_g = dgl.add_self_loop(train_g)

"""## Model definition

Here we build two models from the existing implementation in the DGL library. Both models consists of 2 convolutional layers. 

`GCN` is a standard graph convolutional network, and serves as a baseline. `GraphSAGE` is a more advanced and powerful network that we're testing.

We also define the `DotPredictor`, which helps to predict the likelihood of whether an edge exists or not, by taking the dot product of two nodes. 
"""

from dgl.nn import SAGEConv
from dgl.nn import GraphConv

# ----------- create model -------------- #
# build an ordinary GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

"""## Training and testing pipeline

Here, we define a pipeline for training our model, and evaluating its performance.

For evaluation metric, we use AUC, which is explained in more detail in the blog post. 

---
Note: In the blog post, we mentioned using ranking-based metrics such as `HitRate@K`. However, such kind of metric cannot be used in our colab implementation, since we randomly split the testing edges from the training edges. This means that some users will not have an friend in the test set, and they will never receive a hit when measuring `HitRate@K`. To solve this, we could have tried to split our dataset in a better way, if given more time.
"""

def pipeline(model_name='GCN', hidden_size=16):
    # model_name can be GCN or SAGE
    # hidden_size is the size of the hidden layer in the neural net
    if model_name == 'GCN':
        model = GCN(train_g.ndata['feat'].shape[1], hidden_size)
    elif model_name == 'SAGE':
        model = GraphSAGE(train_g.ndata['feat'].shape[1], hidden_size)
        
    pred = DotPredictor()

    def compute_loss(pos_score, neg_score):  # computes the loss based on binary cross entropy
        scores = torch.cat([pos_score, neg_score])
        labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
        return F.binary_cross_entropy_with_logits(scores, labels)

    def compute_auc(pos_score, neg_score):  # computes AUC (Area-Under-Curve) score
        scores = torch.cat([pos_score, neg_score]).numpy()
        labels = torch.cat(
            [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
        return roc_auc_score(labels, scores)


    # ----------- set up loss and optimizer -------------- #
    # in this case, loss will in training loop
    optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

    # ----------- training -------------------------------- #
    all_logits = []
    for e in range(200):
        # forward
        h = model(train_g, train_g.ndata['feat'])  # get node embeddings
        pos_score = pred(train_pos_g, h)
        neg_score = pred(train_neg_g, h)
        loss = compute_loss(pos_score, neg_score)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {}'.format(e, loss))

    # ----------- test and check results ---------------- #
    from sklearn.metrics import roc_auc_score
    with torch.no_grad():
        pos_score = pred(test_pos_g, h)
        neg_score = pred(test_neg_g, h)
        print('AUC', compute_auc(pos_score, neg_score))
    
    return h  # return node embeddings

"""### Training and testing GCN:"""

h = pipeline("GCN")

"""### Training and testing SAGE:"""

h1 = pipeline("SAGE")

"""# Friend recommendation

Once we trained our model, we can finally suggest friends. To do this, we calculate the embedding dot products between a given user and all other users that he/she is currently NOT friends with. Then we choose 5 users with the highest dot product scores to recommend to him/her.
"""

# --------- generate recommendation for user -------- #
num_nodes = len(graph.nodes())
nodes = list(graph.nodes())
def generate_rec(h, user_id=0):
    # `h` represents the node embeddings, with shape [num_nodes, hidden_size]

    # generate a graph with (num_nodes - num_friends_of_user) edges
    # one end of the edge is user_id
    # the other end is a user that's NOT friends with user_id
    user_friends = set()
    user_neg_u, user_neg_v = [], []
    for n1, n2 in zip(u, v):   # get all friends of user_id
        if int(n1) == user_id:
            user_friends.add(int(n2))
        if int(n2) == user_id:
            user_friends.add(int(n1))
    
    for i in range(num_nodes):  # generate "negative edges" for user_id
        if i != user_id and i not in user_friends:
            user_neg_u.append(user_id)
            user_neg_v.append(i)

    user_g = dgl.graph((user_neg_u, user_neg_v), num_nodes=g.number_of_nodes())
    
    pred = DotPredictor()
    
    # calculate the score of each user
    scores = [(i, score) for i, score in enumerate(pred(user_g, h).detach().numpy())]

    # produce final ranked list
    scores.sort(key=lambda x: -x[1])

    # display results
    # print(f"List of 5 suggested friends for user {user_id}:")
    # r = []
    # for i in range(5):
    #   # print(f'- User {scores[i][0]}, score = {scores[i][1]}')
    #   # print(nodes[scores[i][0]])
    #   r.append(nodes[scores[i][0]])
    return scores

"""The recommended users are shown as follows:"""

from tqdm import tqdm

user,item = np.nonzero(T_test)
pred = []
real = []
for i,j in tqdm(zip(user, item),total = len(user)):
    us = nodes.index('user{}'.format(i))
    item = nodes.index('item{}'.format(j))

    temp_rec = np.array(generate_rec(h1, user_id=us))
    try:
      ad = np.where(temp_rec[:,0]==item)[0][0]
      pred.append(temp_rec[ad,1])
    except:
      pred.append(0)
    if T_test[i,j] > 2:
        real.append(1)
    else:
        real.append(0)

from sklearn.metrics import accuracy_score
pred = np.array(pred)
real = np.array(real)
p = pred.copy()

threshold = 1
p[pred <= threshold] = 1
p[pred > threshold] = 0

print(accuracy_score(real,p))

RuntimeError: Could not infer dtype of numpy.int32